In [1]:
import numpy as np
import pandas as pd    
import matplotlib.pyplot as plt

import pykat.optics.maps as pkm
from pykat import finesse                 
from pykat.commands import *               
from pykat.optics.maps import *            
from scipy.special import factorial
import fileinput

%matplotlib inline
pykat.init_pykat_plotting(dpi=90)

                                              ..-
    PyKat 1.2.1           _                  '(
                          \`.|\.__...-""""-_." )
       ..+-----.._        /  ' `            .-'
   . '            `:      7/* _/._\    \   (
  (        '::;;+;;:      `-"' =" /,`"" `) /
  L.        \`:::a:f            c_/     n_'
  ..`--...___`.  .    ,
   `^-....____:   +.      www.gwoptics.org/pykat



## Content:
1. This file is about calculating the layer coefficients from one particular aLIGO measured map 'ETM05_S1_finesse'

2. The layer coefficients calculated here are used to generate random maps in notebook 'Random maps generation with ETM05_S1_finesse.ipynb'('Random' for later)

3. In order to test that the functions in this file and in 'Random' are working properly, a test file 'testing.txt' is generated in 'Random' with the fake layer coefficients [0,0,1,2,3,4,5,6,7,8] and is decomposed in this file, the resulting coefficients are close to [0,0,1,2,3,4,5,6,7,8], see below.

In [2]:
# return a list of Zernike maps(basis) for aLIGO maps to project to
def Zernikes(shape, radius, step_size, max_zern):
    center = (shape-1)/2 # Center of the map
    rrange = radius/step_size  # Range of the map such that the radius is 0.15 for a given step_size
    zernikes=[] # list of Zernike maps

    # This is the same set of functions in the Zernike maps generation notebook
    def theta(x,y):
        phi = np.arctan2(y, x)
        return phi

    def radial(x,y,n,m):
        if m<0:
            m=-m
        sum=0
        for k in range(int((n-m)/2)+1):   
            r=(-1)**k*factorial(n-k)/factorial(k)/factorial((n+m)/2-k)/factorial((n-m)/2-k)*((x**2+y**2)/(rrange**2))**(n/2-k)
            # Here I used '(x**2+y**2)/(rrange**2)' instead of just '(x**2+y**2)' to normalize the radius
            sum+=r
        return sum

    def angular(x,y,n,m): 
        a=theta(x,y)
        if m>=0:
            angular=np.cos(m*a)
        else:
            angular=-np.sin(m*a)
        return angular
    
    for n in range(max_zern):
        for m in range(-n,n+1,2):
            stepRange = np.arange(shape)-center
            x,y=np.meshgrid(stepRange,stepRange,sparse=True)
            zfunc=radial(x,y,n,m)*angular(x,y,n,m)
            for i in range(shape):
                for j in range(shape): 
                    if (i-center)**2+(j-center)**2>= rrange**2:
                        zfunc[i][j]=0  # Set the values outside the cropping radius to zero
            zmap=zfunc/np.abs(zfunc).max() # Such that the amplitude(maximum value in the map data) equals to 1
            zernikes.append(zmap)
    return zernikes # Return a list of Zernike maps

In [3]:
def layerCoeffs(filename, zernikebasis, order=10):
    i=0 # Create a conversion between {n}{m} to {i}
    dic={}
    for n in range(order):
        for m in range(-n,n+1,2):
            dic[f'{n}{m}']=i
            i=i+1

    aLIGO=pd.read_csv(filename, header=None, sep=" ", skiprows=9).dropna(axis=1).values
    
    layerCoeff=[0,0] # Two zeros represent the first two layer coefficients
    for n in range(2,order): # Start from the third layer
        layer=0
        for m in range(-n,n+1,2):
            index=dic[f'{n}{m}']
            coeff=((aLIGO*zernikebasis[index]).sum())/((zernikebasis[index]**2).sum())
            if n==2 and m==0: # The coefficient of Z20 will not be added to the layer coefficient since it can be removed manually
                continue
            else:
                layer+=coeff**2
        layerCoeff.append(np.sqrt(layer))
    return layerCoeff

In [11]:
# Testing of random map generation
# the 'testing.txt' was generated in the random maps generation file with layer coefficients of [0,0,1,2,3,4,5,6,7,8]
layerCoeffs('testing.txt',Zernikes(1131,0.15,0.0002669951063580811,10))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


[0,
 0,
 0.9991220531215063,
 1.9998863264067825,
 2.9992921522027496,
 4.001539932389178,
 4.99919168488248,
 5.999938721040349,
 6.9989494338149845,
 8.002108867401773]

In [4]:
layerCoeffs('aLIGOmeasuredmaps/ETM05_S1_finesse.txt',Zernikes(1131,0.15,0.0002669951063580811,10))

# Use these set of values to generate the random maps

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


[0,
 0,
 0.20190713375335292,
 0.1458974321372058,
 0.13728131907327343,
 0.0647602622253127,
 0.0639643843456696,
 0.06163558192765335,
 0.11786865519746435,
 0.10733282124903537]